In [2]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

65536/57026 [==================================] - 0s 1us/step


In [4]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


In [5]:
print(train_data[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [6]:
print(train_targets)

[15.2 42.3 50.  21.1 17.7 18.5 11.3 15.6 15.6 14.4 12.1 17.9 23.1 19.9
 15.7  8.8 50.  22.5 24.1 27.5 10.9 30.8 32.9 24.  18.5 13.3 22.9 34.7
 16.6 17.5 22.3 16.1 14.9 23.1 34.9 25.  13.9 13.1 20.4 20.  15.2 24.7
 22.2 16.7 12.7 15.6 18.4 21.  30.1 15.1 18.7  9.6 31.5 24.8 19.1 22.
 14.5 11.  32.  29.4 20.3 24.4 14.6 19.5 14.1 14.3 15.6 10.5  6.3 19.3
 19.3 13.4 36.4 17.8 13.5 16.5  8.3 14.3 16.  13.4 28.6 43.5 20.2 22.
 23.  20.7 12.5 48.5 14.6 13.4 23.7 50.  21.7 39.8 38.7 22.2 34.9 22.5
 31.1 28.7 46.  41.7 21.  26.6 15.  24.4 13.3 21.2 11.7 21.7 19.4 50.
 22.8 19.7 24.7 36.2 14.2 18.9 18.3 20.6 24.6 18.2  8.7 44.  10.4 13.2
 21.2 37.  30.7 22.9 20.  19.3 31.7 32.  23.1 18.8 10.9 50.  19.6  5.
 14.4 19.8 13.8 19.6 23.9 24.5 25.  19.9 17.2 24.6 13.5 26.6 21.4 11.9
 22.6 19.6  8.5 23.7 23.1 22.4 20.5 23.6 18.4 35.2 23.1 27.9 20.6 23.7
 28.  13.6 27.1 23.6 20.6 18.2 21.7 17.1  8.4 25.3 13.8 22.2 18.4 20.7
 31.6 30.5 20.3  8.8 19.2 19.4 23.1 23.  14.8 48.8 22.6 33.4 21.1 13.6
 32.2 13.1

In [7]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

# test data를 정규화 할 때도 test set를 사용하면 안된다!
test_data -= mean
test_data /= std

In [11]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

## K-fold cross-vaildation을 사용한 훈련 검증🧨
<br/>
샘플이 적은 경우, 이전과 같이 데이터를 훈련 셋과 검증 셋으로 나누기 부담스럽다.
<p>
이런 상황에 가장 좋은 방법은 k-fold cross-vaildation을 사용하는 것이다.<br/>
쉽게 데이터를 K개로 분할해 K개의 모델을 각각 만들어 K-1의 분할을 해서 평가하는 방법이다.<br/>
검증 점수는 K개의 검증 점수 평균이 된다.</p>

In [12]:
import numpy as np

k = 4
num_val_samples = len(train_data)//k
num_epochs = 100
all_score = []
for i in range(k):
    print('처리중인 폴드 #', i)
    val_data = train_data[i*num_val_samples: (i+1) * num_val_samples]
    val_target = train_targets[i*num_val_samples: (i+1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i*num_val_samples],
        train_data[(i+1)*num_val_samples:]],
        axis=0
    )
    partial_train_target = np.concatenate(
        [train_targets[:i*num_val_samples],
        train_targets[(i+1)*num_val_samples:]],
        axis=0
    )

    model = build_model()
    model.fit(partial_train_data, partial_train_target, epochs=num_epochs, batch_size=1, verbose=1)
    val_mse, val_mae = model.evaluate(val_data, val_target, verbose=1)
    all_score.append(val_mse)

처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3


In [13]:
all_score

[10.11679458618164]

In [14]:
print(all_score)

[10.11679458618164]


In [15]:
num_epochs = 500
all_score = []
all_mae_histories = []
for i in range(k):
    print('처리중인 폴드 #', i)
    val_data = train_data[i*num_val_samples: (i+1) * num_val_samples]
    val_target = train_targets[i*num_val_samples: (i+1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i*num_val_samples],
        train_data[(i+1)*num_val_samples:]],
        axis=0
    )
    partial_train_target = np.concatenate(
        [train_targets[:i*num_val_samples],
        train_targets[(i+1)*num_val_samples:]],
        axis=0
    )

    model = build_model()
    history = model.fit(partial_train_data, partial_train_target, 
                        validation_data=(val_data, val_target),
                        epochs=num_epochs, batch_size=1, verbose=1)
    mae_history = history.history['val_mean_absoulte_error']
    all_mae_histories.append(mae_history)

처리중인 폴드 # 0
Epoch 1/500
303/303 [==============================] - 2s 5ms/step - loss: 205.9149 - mae: 10.7700 - val_loss: 39.3426 - val_mae: 3.9463
Epoch 2/500
303/303 [==============================] - 1s 4ms/step - loss: 34.1216 - mae: 3.9898 - val_loss: 24.1098 - val_mae: 3.2488
Epoch 3/500
303/303 [==============================] - 1s 5ms/step - loss: 24.0691 - mae: 3.2968 - val_loss: 19.5830 - val_mae: 2.8664
Epoch 4/500
303/303 [==============================] - 1s 4ms/step - loss: 20.3483 - mae: 3.0218 - val_loss: 16.7464 - val_mae: 2.5913
Epoch 5/500
303/303 [==============================] - 1s 4ms/step - loss: 17.8615 - mae: 2.7994 - val_loss: 14.7206 - val_mae: 2.4979
Epoch 6/500
303/303 [==============================] - 1s 4ms/step - loss: 15.6415 - mae: 2.6601 - val_loss: 13.3107 - val_mae: 2.3075
Epoch 7/500
303/303 [==============================] - 1s 4ms/step - loss: 15.1980 - mae: 2.5592 - val_loss: 13.8370 - val_mae: 2.3634
Epoch 8/500
303/303 [====================